<a href="https://colab.research.google.com/github/dspark0419/DATA-205/blob/main/Project_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy import random

In [2]:
!pip install --upgrade plotly

import plotly.express as px
import plotly.graph_objects as go

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


In [3]:
import json
import requests

In [4]:
!pip install --upgrade geopandas
!pip install adjustText

import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import adjustText as aT

Requirement already up-to-date: geopandas in /usr/local/lib/python3.7/dist-packages (0.9.0)


When plotting choropleth maps, have to specify 4 necessary elements:
* The GeoJSON file with the id keys or featureidkey
* The DataFrame object which contains our data
* The column in the DataFrame which make the link between the GeoJSON data
* The column of the DataFrame that will give the intensity of the color. Basically, it is the indicator we want to see on the map.

# **MC Population by Zip Codes**

## **MC Zip Codes**

In [5]:
mcZipMap = gpd.read_file('ZipCode_2017.shp')

Remove the zip codes outside the county.

In [6]:
zip_out = [336, 404, 413, 414, 416, 702, 707, 807, 809, 811, 812, 814, 815, 843, 851]
mcZip = mcZipMap[~mcZipMap['OBJECTID'].isin(zip_out)]

In [7]:
# I think the bottom-right zip code 20993 is 20903.
mcZip['ZIPCODE'].replace({20993: 20903}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Remove the zip codes for P.O. Box and Single Entities.

In [8]:
pobox = [20813, 20824, 20825, 20827, 20830, 20847, 20848, 20849, 20859, 20875, 20880, 20883, 20884, 20885, 20896,
         20891, 20898, 20907, 20908, 20911, 20913, 20914, 20915, 20916, 20918] #20880, 20896

In [9]:
single_entity = [20810, 20811, 20857, 20889, 20892, 20894, 20897, 20899, 20993, 20997] #20889, 20899, 20892

In [10]:
nonstd_zip = pobox + single_entity

In [11]:
std_zip = mcZip[~mcZip['ZIPCODE'].isin(nonstd_zip)].ZIPCODE.unique()
std_zip

array([20868, 20812, 20905, 20815, 20814, 20872, 20895, 20852, 20877,
       20707, 20878, 20879, 20874, 20842, 21797, 20882, 21771, 20904,
       20886, 20871, 20861, 20816, 20903, 20817, 20818, 20901, 20854,
       20851, 20850, 20906, 20866, 20853, 20777, 20860, 20832, 20855,
       20862, 20838, 20876, 20833, 20910, 20902, 20841, 20837, 20912,
       20839])

In [12]:
mcZip_std = mcZip[mcZip['ZIPCODE'].isin(std_zip)].sort_values('ZIPCODE')

In [13]:
mcZip_std['ZIPCODE'].unique()

array([20707, 20777, 20812, 20814, 20815, 20816, 20817, 20818, 20832,
       20833, 20837, 20838, 20839, 20841, 20842, 20850, 20851, 20852,
       20853, 20854, 20855, 20860, 20861, 20862, 20866, 20868, 20871,
       20872, 20874, 20876, 20877, 20878, 20879, 20882, 20886, 20895,
       20901, 20902, 20903, 20904, 20905, 20906, 20910, 20912, 21771,
       21797])

## **MC Population by Zip Codes**

In [14]:
mcZcta = pd.read_csv('mcZcta.csv')
mcZcta.head()

,Unnamed: 0,OBJECTID_1,ZCTA5CE10,FIRST_STAT,FIRST_GEOI,FIRST_CLAS,FIRST_MTFC,FIRST_FUNC,ZCTA5N,STATE,AREALAND,AREAWATR,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,POP65_,PCTPOP65_,MEDAGE,VACNS,PVACNS,PHOWN,PWOMORT,PRENT,PLT18SP,REPORT_2_P,REPORT_9_P,Shape_Length,Shape_Area
0,62,63,20707,24,2420707,B5,G6350,S,20707,24,28896187,481328,31538,13576,9599,13425,72,2849,15,111,796,4671,30.4,42.6,0.2,9.0,0.0,0.4,2.5,14.8,2998,9.5,35.7,892,6.6,53.6,12.5,38.7,32.6,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/census2010/PDF/0...,44625.382501,4.827461e+07
1,102,103,20777,24,2420777,B5,G6350,S,20777,24,19568844,110664,3314,1111,2692,155,10,274,2,2,79,100,81.2,4.7,0.3,8.3,0.1,0.1,2.4,3.0,428,12.9,45.6,27,2.4,76.3,20.1,4.5,7.7,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/census2010/PDF/0...,33086.301965,3.274726e+07
2,111,112,20812,24,2420812,B5,G6350,S,20812,24,496734,371058,255,100,230,3,0,6,0,0,6,10,90.2,1.2,0.0,2.4,0.0,0.0,2.4,3.9,33,12.9,42.4,4,4.0,72.9,19.5,9.4,4.3,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/census2010/PDF/0...,5468.269608,1.437529e+06
3,112,113,20814,24,2420814,B5,G6350,S,20814,24,13166146,52953,27642,14103,21021,1134,34,2572,20,74,695,2092,76.0,4.1,0.1,9.3,0.1,0.3,2.5,7.6,4508,16.3,40.3,838,5.9,40.5,27.7,43.9,14.4,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/census2010/PDF/0...,32694.708048,2.192136e+07
4,113,114,20815,24,2420815,B5,G6350,S,20815,24,14066048,35097,29082,13850,23357,1256,28,1804,13,90,702,1832,80.3,4.3,0.1,6.2,0.0,0.3,2.4,6.3,6002,20.6,45.4,759,5.5,45.4,28.9,36.2,13.5,http://mdpgis.mdp.state.md.us/Census2010/PDF/0...,http://mdpgis.mdp.state.md.us/census2010/PDF/0...,30177.706413,2.337013e+07


In [15]:
mcZcta = mcZcta.loc[:, ~mcZcta.columns.str.contains('^Unnamed')]

In [16]:
mcZcta.columns

Index(['OBJECTID_1', 'ZCTA5CE10', 'FIRST_STAT', 'FIRST_GEOI', 'FIRST_CLAS',
       'FIRST_MTFC', 'FIRST_FUNC', 'ZCTA5N', 'STATE', 'AREALAND', 'AREAWATR',
       'POP100', 'HU100', 'NHW', 'NHB', 'NHAI', 'NHA', 'NHNH', 'NHO', 'NHT',
       'HISP', 'PNHW', 'PNHB', 'PNHAI', 'PNHA', 'PNHNH', 'PNHO', 'PNHT',
       'PHISP', 'POP65_', 'PCTPOP65_', 'MEDAGE', 'VACNS', 'PVACNS', 'PHOWN',
       'PWOMORT', 'PRENT', 'PLT18SP', 'REPORT_2_P', 'REPORT_9_P',
       'Shape_Length', 'Shape_Area'],
      dtype='object')

In [17]:
pop_col = ['ZCTA5N', 'AREALAND', 'AREAWATR', 'POP100', 'HU100', 'NHW', 'NHB', 'NHAI', 'NHA', 'NHNH', 
           'NHO', 'NHT', 'HISP', 'PNHW', 'PNHB', 'PNHAI', 'PNHA', 'PNHNH', 'PNHO', 'PNHT', 'PHISP', 
           'POP65_', 'PCTPOP65_', 'Shape_Area']
mcPop = mcZcta[pop_col]
mcPop.head()

,ZCTA5N,AREALAND,AREAWATR,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,POP65_,PCTPOP65_,Shape_Area
0,20707,28896187,481328,31538,13576,9599,13425,72,2849,15,111,796,4671,30.4,42.6,0.2,9.0,0.0,0.4,2.5,14.8,2998,9.5,4.827461e+07
1,20777,19568844,110664,3314,1111,2692,155,10,274,2,2,79,100,81.2,4.7,0.3,8.3,0.1,0.1,2.4,3.0,428,12.9,3.274726e+07
2,20812,496734,371058,255,100,230,3,0,6,0,0,6,10,90.2,1.2,0.0,2.4,0.0,0.0,2.4,3.9,33,12.9,1.437529e+06
3,20814,13166146,52953,27642,14103,21021,1134,34,2572,20,74,695,2092,76.0,4.1,0.1,9.3,0.1,0.3,2.5,7.6,4508,16.3,2.192136e+07
4,20815,14066048,35097,29082,13850,23357,1256,28,1804,13,90,702,1832,80.3,4.3,0.1,6.2,0.0,0.3,2.4,6.3,6002,20.6,2.337013e+07


# **Chopleth Mapbox for MC Population by Zip Codes**

In [18]:
mcZip_std.head()

,OBJECTID,ZIPCODE,POSTAL,STATE,SDEZIPCODE,geometry
35,694,20707,LAUREL,MD,2.188329e+07,"POLYGON ((-76.88776 39.13176, -76.89107 39.127..."
77,831,20777,HIGHLAND,MD,5.643254e+06,"POLYGON ((-77.00054 39.16932, -77.00038 39.169..."
6,412,20812,GLEN ECHO,MD,2.387338e+05,"POLYGON ((-77.14515 38.96322, -77.14542 38.963..."
10,705,20812,GLEN ECHO,MD,8.217468e+01,"POLYGON ((-77.13602 38.96457, -77.13602 38.964..."
14,673,20812,GLEN ECHO,MD,9.800011e+06,"POLYGON ((-77.14565 38.97274, -77.14540 38.972..."


In [19]:
mcZip_std[mcZip_std['ZIPCODE']==20878]

,OBJECTID,ZIPCODE,POSTAL,STATE,SDEZIPCODE,geometry
95,871,20878,GAITHERSBURG,MD,5.665708e+08,"POLYGON ((-77.23599 39.16218, -77.23592 39.162..."
36,695,20878,GAITHERSBURG,MD,3.493518e+03,"POLYGON ((-77.21251 39.14052, -77.21260 39.140..."


Combine the polygons of the same zip code in the geoemtry column 

In [20]:
mcZip_multiPoly = mcZip_std.dissolve(by='ZIPCODE').reset_index()
mcZip_multiPoly.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ZIPCODE     46 non-null     int64   
 1   geometry    46 non-null     geometry
 2   OBJECTID    46 non-null     int64   
 3   POSTAL      46 non-null     object  
 4   STATE       46 non-null     object  
 5   SDEZIPCODE  46 non-null     float64 
dtypes: float64(1), geometry(1), int64(2), object(2)
memory usage: 2.3+ KB


In [21]:
mcZip_multiPoly[mcZip_multiPoly['ZIPCODE']==20878]

,ZIPCODE,geometry,OBJECTID,POSTAL,STATE,SDEZIPCODE
31,20878,"MULTIPOLYGON (((-77.23599 39.16218, -77.23592 ...",871,GAITHERSBURG,MD,5.665708e+08


Create a geojson file for the data mcZip_multiPoly

In [22]:
mcZip_multiPoly.to_file(driver ='GeoJSON', filename='mcZip_multiPoly.geojson')

In [23]:
with open('mcZip_multiPoly.geojson') as file:
  mcZip_multiPoly_geoj = json.load(file)

Merge the two datasets mcZip_multiPoly and mcZcta using the zip codes from the dataset mcZcta.

In [24]:
mcZipPop = pd.merge(mcZip_multiPoly, mcPop, left_on='ZIPCODE', right_on='ZCTA5N', how='right')
mcZipPop.head()

,ZIPCODE,geometry,OBJECTID,POSTAL,STATE,SDEZIPCODE,ZCTA5N,AREALAND,AREAWATR,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,POP65_,PCTPOP65_,Shape_Area
0,20707,"POLYGON ((-76.88776 39.13176, -76.89107 39.127...",694,LAUREL,MD,2.188329e+07,20707,28896187,481328,31538,13576,9599,13425,72,2849,15,111,796,4671,30.4,42.6,0.2,9.0,0.0,0.4,2.5,14.8,2998,9.5,4.827461e+07
1,20777,"POLYGON ((-77.00054 39.16932, -77.00038 39.169...",831,HIGHLAND,MD,5.643254e+06,20777,19568844,110664,3314,1111,2692,155,10,274,2,2,79,100,81.2,4.7,0.3,8.3,0.1,0.1,2.4,3.0,428,12.9,3.274726e+07
2,20812,"POLYGON ((-77.14515 38.96322, -77.14542 38.963...",412,GLEN ECHO,MD,2.387338e+05,20812,496734,371058,255,100,230,3,0,6,0,0,6,10,90.2,1.2,0.0,2.4,0.0,0.0,2.4,3.9,33,12.9,1.437529e+06
3,20814,"MULTIPOLYGON (((-77.09643 38.99518, -77.09648 ...",680,BETHESDA,MD,1.191660e+05,20814,13166146,52953,27642,14103,21021,1134,34,2572,20,74,695,2092,76.0,4.1,0.1,9.3,0.1,0.3,2.5,7.6,4508,16.3,2.192136e+07
4,20815,"MULTIPOLYGON (((-77.09501 38.98087, -77.09512 ...",865,CHEVY CHASE,MD,1.519956e+08,20815,14066048,35097,29082,13850,23357,1256,28,1804,13,90,702,1832,80.3,4.3,0.1,6.2,0.0,0.3,2.4,6.3,6002,20.6,2.337013e+07


Compute representative points for zip codes

In [25]:
mcZipPop['center'] = mcZipPop['geometry'].representative_point()

Compute the area of the region for each zip cose.

In [26]:
for index, row in mcZipPop.iterrows():
  # Get the area of the polygon
  poly_area = row['geometry'].area

mcZipPop['Area'] = mcZipPop.area

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """


In [27]:
mcZipPop.head()

,ZIPCODE,geometry,OBJECTID,POSTAL,STATE,SDEZIPCODE,ZCTA5N,AREALAND,AREAWATR,POP100,HU100,NHW,NHB,NHAI,NHA,NHNH,NHO,NHT,HISP,PNHW,PNHB,PNHAI,PNHA,PNHNH,PNHO,PNHT,PHISP,POP65_,PCTPOP65_,Shape_Area,center,Area
0,20707,"POLYGON ((-76.88776 39.13176, -76.89107 39.127...",694,LAUREL,MD,2.188329e+07,20707,28896187,481328,31538,13576,9599,13425,72,2849,15,111,796,4671,30.4,42.6,0.2,9.0,0.0,0.4,2.5,14.8,2998,9.5,4.827461e+07,POINT (-76.91335 39.11759),0.000212
1,20777,"POLYGON ((-77.00054 39.16932, -77.00038 39.169...",831,HIGHLAND,MD,5.643254e+06,20777,19568844,110664,3314,1111,2692,155,10,274,2,2,79,100,81.2,4.7,0.3,8.3,0.1,0.1,2.4,3.0,428,12.9,3.274726e+07,POINT (-76.99845 39.16212),0.000055
2,20812,"POLYGON ((-77.14515 38.96322, -77.14542 38.963...",412,GLEN ECHO,MD,2.387338e+05,20812,496734,371058,255,100,230,3,0,6,0,0,6,10,90.2,1.2,0.0,2.4,0.0,0.0,2.4,3.9,33,12.9,1.437529e+06,POINT (-77.14297 38.96671),0.000097
3,20814,"MULTIPOLYGON (((-77.09643 38.99518, -77.09648 ...",680,BETHESDA,MD,1.191660e+05,20814,13166146,52953,27642,14103,21021,1134,34,2572,20,74,695,2092,76.0,4.1,0.1,9.3,0.1,0.3,2.5,7.6,4508,16.3,2.192136e+07,POINT (-77.10057 38.98763),0.001131
4,20815,"MULTIPOLYGON (((-77.09501 38.98087, -77.09512 ...",865,CHEVY CHASE,MD,1.519956e+08,20815,14066048,35097,29082,13850,23357,1256,28,1804,13,90,702,1832,80.3,4.3,0.1,6.2,0.0,0.3,2.4,6.3,6002,20.6,2.337013e+07,POINT (-77.07184 38.98413),0.001468


In [28]:
mcZipPop.to_csv('mcZipPop_multiPoly.csv')

Plot the static representative points for the zip codes

In [29]:
#token = ''

fig = px.choropleth_mapbox(mcZipPop,
                           geojson=mcZip_multiPoly_geoj,
                           featureidkey='properties.ZIPCODE',
                           locations='ZIPCODE',
                           color='POP100',
                           mapbox_style='carto-positron',
                           color_continuous_scale='viridis',
                           center={'lon': -77.195434, 'lat': 39.139551},
                           zoom=9,
                           opacity=0.7,
                           hover_data=['ZIPCODE', 'Area', 'POP100'],
                           labels={'ZIPCODE':'Zip', 'POP100':'Pop', 'Area':'Area'}
                           )

fig.add_trace(go.Scattermapbox(lon=mcZipPop.center.x,
                               lat=mcZipPop.center.y,
                               mode='markers',
                               marker=dict(size=5, color='black', opacity=0.7), # or marker=go.scattermapbox.Marker(size=5, color='black', opacity=0.7)
                               text=mcZipPop.ZIPCODE,
                               hoverinfo='skip', # or 'none'
                               #hoverinfo='text'
                               ))

#fig.update_layout(mapbox_style='dark', mapbox_accesstoken=token)

fig.show()

# **Dash**

In [30]:
!pip install --upgrade jupyter-dash

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

Requirement already up-to-date: jupyter-dash in /usr/local/lib/python3.7/dist-packages (0.4.0)


In [31]:
Filter = np.array(['Area', 'POP100'])

app = JupyterDash(__name__)
app.layout = html.Div([
                        html.H2('MC Map with Dropbox', style={'text-align':'center'}),
                        html.P('Filter', style={'font-weight': 'bold', 'margin-left': '4em'}),
                        dcc.Dropdown(id='filter-dropdown',
                                     value='POP100',
                                     #placeholder='Select',
                                     options=[{'label': c, 'value': c} for c in Filter],
                                     style={'float': 'left', 'width': '40%', 'margin-left': '2em'},
                                     #multi=True
                                     ),
                        html.Div(id='info-selected',
                                 style={'font-weight': 'bold', 'margin-left': '4em'}
                                 ),
                        dcc.Graph(id='zip_pop',
                                  style={'height': '100%', 'width': '70%', 'margin-left': '20em'}
                                  )
                        ])

@app.callback([Output('info-selected', 'children'), Output('zip_pop', 'figure')],
               Input('filter-dropdown', 'value')
               )

def display_choropleth(value):
  info = 'Information of Zip Codes by {}'.format(value)

  fig = px.choropleth_mapbox(mcZipPop,
                           geojson=mcZip_multiPoly_geoj,
                           featureidkey='properties.ZIPCODE',
                           locations='ZIPCODE',
                           color=mcZipPop[value],
                           mapbox_style='carto-positron',
                           color_continuous_scale='viridis',
                           center={'lon': -77.195434, 'lat': 39.139551},
                           zoom=9,
                           opacity=0.7,
                           hover_data=['ZIPCODE', 'Area', 'POP100'],
                           labels={'ZIPCODE':'Zip', 'POP100':'Pop', 'Area':'Area'}
                           )

  fig.add_trace(go.Scattermapbox(lon=mcZipPop.center.x,
                               lat=mcZipPop.center.y,
                               mode='markers',
                               marker=dict(size=5, color='black', opacity=0.7),
                               #text=mcZipPop.ZIPCODE,
                               hoverinfo='skip',
                               ))

  fig.update_layout(margin={"r":9,"t":9,"l":9,"b":9})

  return info, fig

if __name__ == '__main__':
  app.run_server(debug=True)

Dash app running on:


<IPython.core.display.Javascript object>

Display hover information

In [32]:
app = JupyterDash(__name__)

styles = {'pre': {'border': 'thin lightgrey solid', 'overflowX': 'scroll'}}

fig = px.choropleth_mapbox(mcZipPop,
                           geojson=mcZip_multiPoly_geoj,
                           featureidkey='properties.ZIPCODE',
                           locations='ZIPCODE',
                           color='POP100',
                           mapbox_style='carto-positron',
                           color_continuous_scale='viridis',
                           center={'lon': -77.195434, 'lat': 39.139551},
                           zoom=9,
                           opacity=0.7,
                           hover_data=['ZIPCODE', 'Area', 'POP100'],
                           labels={'ZIPCODE':'Zip', 'POP100':'Pop', 'Area':'Area'}
                           )

app.layout = html.Div(
    [
     dcc.Graph(id='zip-choropleth', figure=fig),
          
     html.Div(children=
         [
          html.Div(
              [dcc.Markdown("""
                      **Click Data:** Click over regions in the graph.
                      """),
               html.Pre(id='click-data', style=styles['pre'])
               ])
          ]),
     ])

# change clickData by hoverData to display the hover data                           
@app.callback(Output('click-data', 'children'),
               Input('zip-choropleth', 'clickData'))

def display_click_data(clickData):
  return json.dumps(clickData, indent=2)

if __name__ == '__main__':
  app.run_server(debug=True) #mode='inline'

Dash app running on:


<IPython.core.display.Javascript object>

In [33]:
mcZipPop.columns

Index(['ZIPCODE', 'geometry', 'OBJECTID', 'POSTAL', 'STATE', 'SDEZIPCODE',
       'ZCTA5N', 'AREALAND', 'AREAWATR', 'POP100', 'HU100', 'NHW', 'NHB',
       'NHAI', 'NHA', 'NHNH', 'NHO', 'NHT', 'HISP', 'PNHW', 'PNHB', 'PNHAI',
       'PNHA', 'PNHNH', 'PNHO', 'PNHT', 'PHISP', 'POP65_', 'PCTPOP65_',
       'Shape_Area', 'center', 'Area'],
      dtype='object')

In [39]:
colorscale = np.array(['Null', 'POP100', 'Area'])
barpie_col = np.array(['POP100', 'NHW', 'NHB', 'NHA', 'HISP'])

app = JupyterDash(__name__)

app.layout = html.Div(
    [
     html.H1('MC Population (2010)',
             style={'textAlign': 'center', 'color': 'blue', 'backgroundColor': 'black'}),
     
     html.Div(
         [
          html.Div(
              [
               html.Label(['Colorscale',
                           dcc.Dropdown(id='colorscale',
                                        options=[{'label': i, 'value': i} for i in colorscale],
                                        value='Null',
                                        #placeholder='Select',
                                        #multi=True
                                        ),
                           ]),
              ],
              style={'width': '40%', 'display': 'inline-block', 'color': 'blue', 'fontWeight': 'bold',
                     'margin-left': '1em'}
              ),
          
          html.Div(id='info1',
                   style={'font-weight': 'bold', 'margin-left': '1em'}),
          
          dcc.Graph(id='zip-choropleth'),
          
          # html.Div(
          #     [
          #      html.Label(['Slide for a Year',
          #                  dcc.Slider(id='model-year',
          #                             min=mpg['model_year'].min(),
          #                             max=mpg['model_year'].max(),
          #                             value=mpg['model_year'].min(),
          #                             marks={str(year): str(year) for year in mpg['model_year'].unique()},
          #                             step=None,
          #                             ),
          #                  ]),
          #      ],
          #      style={'width': '80%', 'display': 'inline-block', 'fontWeight': 'bold', 'color': 'blue'}
          #      ),
          ],
          style={'width': '45%', 'display': 'inline-block', 'margin-left': '3em'},
          ),
     
     html.Div(
         [
          html.Div(
              [
               html.Label(['Bargraph/Piechart',
                           dcc.RadioItems(id='bar-pie',
                                          options=[{'label': i, 'value': i} for i in barpie_col],
                                          value='POP100',
                                          #style={'margin-top': '0.8em'},
                                          ),
                           ]),
               ],
               style={'width': '70%', 'display': 'inline-block', 'color': 'green', 'fontWeight': 'bold',
                      'margin-left': '3em'}
               ),
          
          html.Div(id='info2',
                   style={'font-weight': 'bold', 'margin-left': '3em'}),
          
          dcc.Graph(id='pop-barpie'),
          ],
          style={'float': 'right', 'width': '45%', 'display': 'inline-block', 'margin-right': '3em'},
          ),
     ])
                        
@app.callback([Output('info1', 'children'), Output('zip-choropleth', 'figure')],
              [Input('colorscale', 'value')])

def update_figure(color_value):
  info1 = 'Click over regions for pie charts'.format()

  if color_value == 'Null':
    fig1 = px.choropleth_mapbox(mcZipPop,
                           geojson=mcZip_multiPoly_geoj,
                           featureidkey='properties.ZIPCODE',
                           locations='ZIPCODE',
                           mapbox_style='carto-positron',
                           color='POP100',
                           color_continuous_scale='viridis',
                           center={'lon': -77.195434, 'lat': 39.139551},
                           zoom=8.9,
                           opacity=0.7,
                           hover_data=['ZIPCODE', 'Area', 'POP100'],
                           labels={'ZIPCODE':'Zip', 'POP100':'Pop', 'Area':'Area'},
                           custom_data=['ZIPCODE', 'Area', 'POP100']
                           )

    fig1.update_layout(transition_duration=10, height=450, margin=dict(l=20, r=20, t=20, b=20))
    
    return info1, fig1

  else:
    fig1 = px.choropleth_mapbox(mcZipPop,
                           geojson=mcZip_multiPoly_geoj,
                           featureidkey='properties.ZIPCODE',
                           locations='ZIPCODE',
                           mapbox_style='carto-positron',
                           color=mcZipPop[color_value],
                           color_continuous_scale='viridis',
                           center={'lon': -77.195434, 'lat': 39.139551},
                           zoom=8.9,
                           opacity=0.7,
                           hover_data=['ZIPCODE', 'Area', 'POP100'],
                           labels={'ZIPCODE':'Zip', 'POP100':'Pop', 'Area':'Area'},
                           custom_data=['ZIPCODE', 'Area', 'POP100']
                           )

    fig1.add_trace(go.Scattermapbox(lon=mcZipPop.center.x,
                               lat=mcZipPop.center.y,
                               mode='markers',
                               marker=dict(size=5, color='black', opacity=0.7),
                               text=mcZipPop.ZIPCODE,
                               hoverinfo='skip',
                               ))
  
    fig1.update_layout(transition_duration=10, height=450, margin=dict(l=20, r=20, t=20, b=20))

    return info1, fig1

@app.callback([Output('info2', 'children'), Output('pop-barpie', 'figure')],
              [Input('colorscale', 'value'), Input('bar-pie', 'value'), Input('zip-choropleth', 'clickData')])

def update_figure(color_value, barpie_value, click_data):
  if color_value == 'Null' or click_data == None:
    info2 = 'Choose a colorscale for a piechart'.format()

    fig2 = px.bar(mcZipPop, x='ZIPCODE', y=barpie_value)

    fig2.update_xaxes(type='category', title='Zip Code', tickangle=-60, tickfont_size=11)

    fig2.update_layout(height=450, margin=dict(l=20, r=20, t=30, b=30))

    return info2, fig2

  else:
    info2 = 'Choose Null in Colorscale for bargraphs'.format()
    #info2 = 'Information of Zip Codes by {}'.format(barpie_value)
    pop_zip = mcZipPop[mcZipPop['ZIPCODE'] == click_data['points'][0]['customdata'][0]]
    pop_zip_sel = pop_zip.loc[:, pop_zip.columns.isin(['NHW', 'NHB', 'NHAI', 'NHA', 'NHNH', 'NHO', 'HISP'])]
    pop_zip_df = pd.melt(pop_zip_sel)
    pop_zip_df.columns = ['Race', 'Pop']

    fig2 = px.pie(pop_zip_df, values='Pop', names='Race', color='Race', hole=0.4,
                  color_discrete_map={'NHW': 'blue', 'NHB': 'orange', 'NHAI': 'grey', 'NHA': 'red',
                                      'NHNH': 'salmon', 'NHO': 'yellow', 'HISP': 'purple'})

    fig2.update_layout(height=400, margin=dict(l=50, r=150, t=50, b=50))

    fig2.update_layout(legend_title_text=' Zip: {}'.format(click_data['points'][0]['customdata'][0]))
    #fig.update_traces(hole=.4, hoverinfo="label+percent+name")

    return info2, fig2

if __name__ == '__main__':
  app.run_server(debug=True) #mode='inline'

Dash app running on:


<IPython.core.display.Javascript object>